In [1]:
import QuantLib as ql
import math as m

# Testing analytic Bates engine against Black formula...

In [2]:
settlementDate = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = settlementDate

In [3]:
dayCounter = ql.ActualActual()
exerciseDate = settlementDate + ql.Period(6,ql.Months)

In [4]:
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 30)

In [5]:
exercise = ql.EuropeanExercise(exerciseDate)

In [6]:
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), 0.1, dayCounter))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), 0.04, dayCounter))

In [7]:
s0 = ql.QuoteHandle(ql.SimpleQuote(32.0))

In [8]:
yearFraction = dayCounter.yearFraction(settlementDate, exerciseDate)

In [9]:
forwardPrice = s0.value()*m.exp((0.1-0.04)*yearFraction)

In [10]:
expected = ql.blackFormula(payoff.optionType(), payoff.strike(), forwardPrice, m.sqrt(0.05*yearFraction)) * m.exp(-0.1*yearFraction)

In [11]:
v0 = 0.05
kappa = 5.0
theta = 0.05
sigma = 1.0e-4
rho = 0.0
lamda = 0.0001
nu = 0.0
delta = 0.0001

In [12]:
option = ql.VanillaOption(payoff, exercise)

In [13]:
process = ql.BatesProcess(riskFreeTS, dividendTS, s0, v0, 
                         kappa, theta, sigma, rho, lamda, nu, delta)

In [14]:
engine = ql.BatesEngine(ql.BatesModel(process), 64)

In [15]:
option.setPricingEngine(engine)

In [16]:
calculated = option.NPV()
calculated

0.8006699946902414

In [17]:
tolerance = 2.0e-7

In [18]:
error = abs(calculated - expected)

In [19]:
if (error > tolerance):
    print("FAILURE")

In [20]:
engine = ql.BatesDetJumpEngine(ql.BatesDetJumpModel(process, 1.0, 0.0001), 64)

In [21]:
option.setPricingEngine(engine)
calculated = option.NPV()
calculated

0.8006699946902414

In [22]:
error = abs(calculated - expected)
if (error > tolerance):
    print("FAILURE")

In [23]:
engine = ql.BatesDoubleExpEngine(ql.BatesDoubleExpModel(process, 0.0001, 0.0001, 0.0001), 64)

In [24]:
option.setPricingEngine(engine)
calculated = option.NPV()
calculated

0.8006699947059968

In [25]:
error = abs(calculated - expected)
if (error > tolerance):
    print("FAILURE")

In [26]:
engine = ql.BatesDoubleExpDetJumpEngine(ql.BatesDoubleExpDetJumpModel( process, 0.0001, 0.0001, 0.0001, 0.5, 1.0, 0.0001), 64)

In [27]:
option.setPricingEngine(engine)
calculated = option.NPV()
calculated

0.8006699947059968

In [28]:
error = abs(calculated - expected)
if (error > tolerance):
    print("FAILURE")

# Testing analytic Bates engine against Merton-76 engine

In [29]:
settlementDate = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = settlementDate

In [30]:
dayCounter = ql.ActualActual()

In [31]:
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 95.0)

In [32]:
riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), 0.1, dayCounter))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), 0.04, dayCounter))

In [33]:
s0 = ql.QuoteHandle(ql.SimpleQuote(100.0))

In [34]:
v0 = 0.0433

In [35]:
vol = ql.SimpleQuote(m.sqrt(v0))

In [36]:
volTS = ql.BlackConstantVol(settlementDate, ql.NullCalendar(), ql.QuoteHandle(vol), dayCounter)

In [37]:
kappa = 0.5
theta = v0
sigma = 1.0e-4
rho = 0.0

In [38]:
jumpIntensity = ql.SimpleQuote(2)
meanLogJump = ql.SimpleQuote(-0.2)
jumpVol = ql.SimpleQuote(0.2)

In [39]:
batesProcess = ql.BatesProcess(riskFreeTS, dividendTS, s0, v0, kappa, theta, sigma, rho,
        jumpIntensity.value(), meanLogJump.value(), jumpVol.value())

In [40]:
mertonProcess = ql.Merton76Process(s0, dividendTS, riskFreeTS,
                                  ql.BlackVolTermStructureHandle(volTS),
                                  ql.QuoteHandle(jumpIntensity),
                                  ql.QuoteHandle(meanLogJump),
                                  ql.QuoteHandle(jumpVol))

In [41]:
batesEngine = ql.BatesEngine(ql.BatesModel(batesProcess), 160)

In [42]:
mcTol = 0.1

In [43]:
mcBatesEngine = ql.MCPREuropeanHestonEngine(batesProcess,
                                           2,
                                           ql.nullInt(),
                                           True,
                                           ql.nullInt(),
                                           mcTol,
                                           ql.nullInt(),
                                           1234)

In [44]:
mertonEngine = ql.JumpDiffusionEngine(mertonProcess, 1e-10, 1000)

In [48]:
for i in (1, 3, 5):
    print(i)
    exerciseDate = settlementDate + ql.Period(i, ql.Years)
    exercise = ql.EuropeanExercise(exerciseDate)
    batesOption = ql.VanillaOption(payoff, exercise)
    batesOption.setPricingEngine(batesEngine)
    calculated = batesOption.NPV()
    print(calculated)
    batesOption.setPricingEngine(mcBatesEngine)
    mcCalculated = batesOption.NPV()
    print(mcCalculated)
    mertonOption = ql.EuropeanOption(payoff, exercise)
    mertonOption.setPricingEngine(mertonEngine)
    expected = mertonOption.NPV()
    
    tolerance = 2e-8
    relError = abs(calculated - expected)/expected
    
    if (relError > tolerance):
        print("FAILURE semianalytic")
        
    mcError = abs(expected - mcCalculated)
    if (relError > 3 * mcTol):
        print("FAILURE MC")    

1
10.817488127412261
10.75704754485689
3
14.788733602117034
14.679900793584162
5
14.972622752712184
14.964592191617832


# Testing analytic Bates engine against Monte-Carlo engine...

In [49]:
settlementDate = ql.Date(30, ql.March, 2007)
ql.Settings.instance().evaluationDate = settlementDate

In [50]:
dayCounter = ql.ActualActual()

In [51]:
exerciseDate = ql.Date(30, ql.March, 2012)

In [52]:
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 100)

In [53]:
exercise = ql.EuropeanExercise(exerciseDate)

In [55]:
# name, v0, kappa, theta, sigma, rho, r, q
hestonModels = [
    # ADI finite difference schemes for option pricing in the 
    # Heston model with correlation, K.J. in t'Hout and S. Foulon,
    ["'t Hout case 1", 0.04, 1.5, 0.04, 0.3, -0.9, 0.025, 0.0],
    # Efficient numerical methods for pricing American options under 
    # stochastic volatility, Samuli Ikonen and Jari Toivanen,
    ["Ikonen-Toivanen", 0.0625, 5, 0.16, 0.9, 0.1, 0.1, 0.0],
    # Not-so-complex logarithms in the Heston model, 
    # Christian Kahl and Peter Jäckel
    ["Kahl-Jaeckel", 0.16, 1.0, 0.16, 2.0, -0.8, 0.0, 0.0],
    # self defined test cases
    ["Equity case", 0.07, 2.0, 0.04, 0.55, -0.8, 0.03, 0.035 ]
]

In [68]:
for heston_model in hestonModels:
    riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), heston_model[6], dayCounter))
    dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), heston_model[7], dayCounter))
    s0 = ql.QuoteHandle(ql.SimpleQuote(100.0))
    
    batesProcess = ql.BatesProcess(riskFreeTS, dividendTS, s0,
                                   heston_model[1], 
                                   heston_model[2], 
                                   heston_model[3], 
                                   heston_model[4], 
                                   heston_model[5], 2.0, -0.2, 0.1
                                  )
    mcTolerance = 0.5
    mcEngine = ql.MCPREuropeanHestonEngine(batesProcess,
                                           ql.nullInt(),
                                           20,
                                           True,
                                           ql.nullInt(),
                                           mcTol,
                                           ql.nullInt(),
                                           1234)
    batesModel = ql.BatesModel(batesProcess)
    fdEngine = ql.FdBatesVanillaEngine(batesModel, 50, 100, 30)
    
    analyticEngine = ql.BatesEngine(batesModel, 160)
    
    option = ql.VanillaOption(payoff, exercise)
    option.setPricingEngine(mcEngine)
    calculated = option.NPV()

    option.setPricingEngine(analyticEngine)
    expected = option.NPV()

    option.setPricingEngine(fdEngine)
    fdCalculated = option.NPV()
    mcError = abs(calculated - expected)
    
    if (mcError > 3*mcTolerance):
        print("FAILURE MC")
        
    fdTolerance = 0.2
    fdError = abs(fdCalculated - expected)
    
    if (fdError > fdTolerance):
        print("FAILURE FD")

# Testing Bates model calibration using DAX volatility data..

In [69]:
settlementDate = ql.Date(5, ql.July, 2002)
ql.Settings.instance().evaluationDate = settlementDate

In [70]:
dayCounter = ql.Actual365Fixed()

In [71]:
calendar = ql.TARGET()

In [72]:
t = [13, 41, 75, 165, 256, 345, 524, 703 ]

In [74]:
r = [0.0357,0.0349,0.0341,0.0355,0.0359,0.0368,0.0386,0.0401]

In [75]:
dates = [settlementDate] + [settlementDate + t_i for t_i in t]

In [87]:
rates = [0.0357] + r

In [90]:
riskFreeTS = ql.YieldTermStructureHandle(ql.ZeroCurve(dates, rates, dayCounter))

In [91]:
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, 0.0, dayCounter))

In [93]:
v =       [ 0.6625,0.4875,0.4204,0.3667,0.3431,0.3267,0.3121,0.3121,
        0.6007,0.4543,0.3967,0.3511,0.3279,0.3154,0.2984,0.2921,
        0.5084,0.4221,0.3718,0.3327,0.3155,0.3027,0.2919,0.2889,
        0.4541,0.3869,0.3492,0.3149,0.2963,0.2926,0.2819,0.2800,
        0.4060,0.3607,0.3330,0.2999,0.2887,0.2811,0.2751,0.2775,
        0.3726,0.3396,0.3108,0.2781,0.2788,0.2722,0.2661,0.2686,
        0.3550,0.3277,0.3012,0.2781,0.2781,0.2661,0.2661,0.2681,
        0.3428,0.3209,0.2958,0.2740,0.2688,0.2627,0.2580,0.2620,
        0.3302,0.3062,0.2799,0.2631,0.2573,0.2533,0.2504,0.2544,
        0.3343,0.2959,0.2705,0.2540,0.2504,0.2464,0.2448,0.2462,
        0.3460,0.2845,0.2624,0.2463,0.2425,0.2385,0.2373,0.2422,
        0.3857,0.2860,0.2578,0.2399,0.2357,0.2327,0.2312,0.2351,
        0.3976,0.2860,0.2607,0.2356,0.2297,0.2268,0.2241,0.2320 ]

In [94]:
s0 = ql.QuoteHandle(ql.SimpleQuote(4468.17))

In [95]:
strike = [3400,3600,3800,4000,4200,4400,
          4500,4600,4800,5000,5200,5400,5600]

In [96]:
v0 = 0.0433

In [97]:
vol = ql.SimpleQuote(m.sqrt(v0))

In [100]:
volTS = ql.BlackConstantVol(settlementDate, ql.NullCalendar(), ql.QuoteHandle(vol), dayCounter)

In [101]:
kappa = 1.0
theta = v0
sigma = 1.0
rho = 0.0
lamda = 1.1098
nu = -0.1285
delta = 0.1702

In [102]:
process = ql.BatesProcess(riskFreeTS, dividendTS, s0, v0, 
                         kappa, theta, sigma, rho, lamda, nu, delta)

In [103]:
batesModel = ql.BatesModel(process)

In [105]:
batesEngine = ql.BatesEngine(batesModel, 64)

In [106]:
options = []

In [114]:
for s in range(0, 13):
    for m in range(0, 8):
        vol = ql.QuoteHandle(ql.SimpleQuote(v[s*8+m]))
        maturity = ql.Period(int((t[m]+3)/7.), ql.Weeks)
        options += [ql.HestonModelHelper(maturity, calendar,
                                          s0.value(), strike[s], vol,
                                          riskFreeTS, dividendTS, 
                                          ql.BlackCalibrationHelper.ImpliedVolError)]
for option in options:
    option.setPricingEngine(batesEngine)

In [115]:
om = ql.LevenbergMarquardt()

In [126]:
batesModel.calibrate(options, om, ql.EndCriteria(400, 40, 1.0e-8, 1.0e-8, 1.0e-8))

In [117]:
expected = 36.6

In [120]:
def getCalibrationError(options):
    sse = 0
    for option in options:
        diff = option.calibrationError()*100.0
        sse += diff*diff
    return sse

In [127]:
calculated = getCalibrationError(options)
calculated

116.49556165090165

In [123]:
if (abs(calculated - expected) > 2.5):
    print("FAILURE")

FAILURE
